# EEG - Flow

## 2. Bridges and auto bads channels

Determine gel bridges and save the gel bridges topography. Interpolate gel bridges if possible. Mark bad channels with the PREP pipeline.

Last edit: 21.06.2023 18:57
@anguyen

In [2]:
import multiprocessing as mp
import time
from itertools import product

from mne import set_log_level as set_log_level_mne
from mne.viz import set_browser_backend

from eeg_flow import set_log_level
from eeg_flow.utils import parallel
from eeg_flow.tasks import bridges_and_autobads

2024-04-19 16:10:39,170 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-04-19 16:10:39,172 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [3]:
set_log_level_mne("WARNING")
set_log_level("INFO")

%matplotlib qt
set_browser_backend("qt")

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [6]:
PARTICIPANTS_WITH_GROUPS: list[str] = ["S24-G6"]  # list of str "Pxx-Gy", e.g. ["P02-G2"]
TASKS: list[str]                    = ["oddball","UT"]  # ["oddball"], ["UT"] or ["oddball", "UT"]
RUNS: list[int]                     = [1,2]  # [1], [2] or [1, 2]

inputs = [
    (t[0].split("-") + list(t[1:]))
    for t in product(PARTICIPANTS_WITH_GROUPS, TASKS, RUNS)
]
print(inputs)

[['S24', 'G6', 'oddball', 1], ['S24', 'G6', 'oddball', 2], ['S24', 'G6', 'UT', 1], ['S24', 'G6', 'UT', 2]]


This step is divided in:
- Visualization of gel-bridges and rejection of the dataset if required
- Interpolation of gel-bridges
- Annotation of bad channels with the PREP pipeline

The created derivatives are:
- Plot of the gel-bridges (`_bridges.svg`)
- Raw with interpolated gel-bridged channels (`_with-bads_raw.fif`)

More information about gel bridgs in this [MNE tutorial](https://mne.tools/stable/auto_examples/preprocessing/eeg_bridging.html).

In [ ]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

assert len(inputs) != 0  # sanity-check
n_jobs = min(len(inputs), mp.cpu_count() - 2)
parallel(bridges_and_autobads, n_jobs, inputs)

Start time 16:33:39


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [02:09<00:40, 40.30s/it]